# Project 4: Practical Matters

We're not going to be learning any new machine learning algorithms in this notebook. Instead, we're going to focus in on a few practical concerns that will allow us to really improve our machine learning efficiency. These aren't giant topics, but they were a bit too large to squeeze into the middle of another notebook. A brief overview of the topics we'll be covering:

- **Image Preprocessing and Data Generation**: Image preprocessing has a few uses in ML. 
    1. Altering an image shapes in order to feed it into our network. 
    2. Distorting images slightly in order to create new, artificial data.
- **Using Pre-trained models**: We can take a model that someone else has trained, make slight alterations, and use it for our own purposes.
- **Model visualization with Tensorboard**: Visualize our model to track performance and aid in tuning.
- **Exporting for use in JavaScript**: We're going to export our models for use with TensorflowJS.

## Image Preprocessing and Data Generation

There's 2 useful scenarios where we'd want to know about data pre-processing:

1. Let's say we create a handwritten digit recognizer NN using MNIST. We want to take that model and turn it into an app. Users can write a number, take a picture of it, and it tells them the number that they wrote. Our input will be smart phone resolution, color picture. We need to feed into a model that's expecting 28x28, grey scale picture. We can accomplish this with Kera's built-in preprocessing.image utility. We can reshape and alter the image however we want.

2. Next, we have an idea for an awesome new app that can identify your dog, and only your dog. You point your phone at something at it says 'Your Dog' or 'Not Your Dog'. This is simple enough generally, but we have a problem: You, like any reasonable person, don't have tens of thousands of pictures of your dog laying around. You can use image preprocessing techniques to generate tens of thousands of artificial pictures of your dog, using your existing pictures These artificial pictures can be used to train a model that is, in fact, able to identify your dog.

Although the above scenarios seem distinct, we use data pre-processing techniques to generate the new data. Image resclaing can easily be demonstrated through artificial image generation.

We're going to be expanding on this later, but this section of the notebook is going to follow [this Keras blog post](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html) very closely.

For this notebook we're going to use a [Dogs vs. Cats dataset](https://www.kaggle.com/c/dogs-vs-cats/data). The original dataet comes with 25K traning images (12,500 images per class). We're going to: 

1. Train a nerural network with all of the data.
2. Train on the 2K images dataset, but generate artifical data from those 2K images.
3. Train on all of the data, using artifical data AND a pre-trained network.

First, we're going to restructure our data into folders. Rather than directly labeling picutres, we're going to sort them into cats and dogs directories.

All you need to do is place the downlaoded data into the same directory as this notebook and run the below cell (it's expecting the zip to be named 'all.zip'). This will delete you zip once it's done, so back it up or comment out the removal line if you want to keep it.

In [2]:
import os
import zipfile

# We don't want to mess with this is we already have the dir.
if not os.path.exists('cvd_data'):
    # Create dir structure
    os.makedirs('cvd_data')
    os.makedirs('cvd_data/train/cats')
    os.makedirs('cvd_data/train/dogs')
    os.makedirs('cvd_data/validation/cats')
    os.makedirs('cvd_data/validation/dogs')
        
    # Extract 2 layers of zip
    with zipfile.ZipFile('all.zip', 'r') as zip_ref:
        zip_ref.extractall('cvd_data')
    
    with zipfile.ZipFile('cvd_data/train.zip', 'r') as zip_ref:
        zip_ref.extractall('cvd_data/train')
    
    # List of all of our pictures
    training_data = os.listdir('cvd_data/train/train')
    
    # Picute names start with 'dog' or 'cat' - sort into seperate lists
    dog_pics = [pic for pic in training_data if 'dog' in pic]
    cat_pics = [pic for pic in training_data if 'cat' in pic]

    # Move all pictures into their sorted directories
    for pic_list in [dog_pics, cat_pics]:
        for pic in pic_list:
            index = pic_list.index(pic)
            # Sorts first 10K into train and last 2500 into validation
            path = 'train' if index < 10000 else 'validation'
            label = 'dogs' if 'dog' in pic else 'cats'
            source = f'cvd_data/train/train/{pic}'
            dest = f'cvd_data/{path}/{label}/{pic}'
            os.rename(source, dest)
    
    # Cleanup. Edit as needed to keep what you want.
    os.remove('all.zip')
    os.remove('cvd_data/train.zip')
    os.remove('cvd_data/test1.zip')
    os.remove('cvd_data/sampleSubmission.csv')
    os.removedirs('cvd_data/train/train')
    

## Model Time

In order to establish our baseline, we're going to run train a model on all of the data. First we're going to create our data generators so that we can reshpae some basic values (we aren't actually generating extra images at this time). We don't technically need to use a data generator, but I'd like for our model trainings to be similar.

Most of the new stuff here is the data generator pieces. Keras can just run `.fit()` with data generators. We're switching to `fit_generator()`. Notice that the first convolutional input is different that our first convolutional layer (from project 2). This is because these picutres are 150x150x3 (3 for RGB values) vs 28x28x1.

I am going to take this opportunity to introduce a new ML concept: **Early stopping**. I'm having issues with overfitting on this training. The validation accuracy keeps dropping by as much as 8% by the end training. We're going to tell Keras to stop training when it sees the validation loss going down.

There's 2 more details here:
1. Callbacks: Callbacks are additional functions that we want to pass at training time. This is how we accomplish early stopping, and it's how we'll feed our traning data into tensorboard later in this notebook.
2. We're setting the `patience` to 2. This is how many epochs we wait before we call it. We expect some bouncing, so we want to change the default (which is zero).
3. We aren't using 'batch size' the normal way in `fit_generator()`. Batch size is the size of the batch to run out of the total dataset. With our generator we can generate infinite data, so `batch_size` doesn't make sense as a parameter. Hence, `steps_per_epoch` and `validation_steps`. All of that said: We aren't actually generating novel images in the first model, so we're setting the batch size based on the amount of images we have in the dataset.

Side note: We aren't going to worry about it here, but another really useful callback is `ModelCheckpoint`. This will save your model at intervals. If you're doing a really big training...or on a Windows PC...then random reboots are a serious threat that can be addressed.

In [1]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
# This is all the new, shiny image preprocessing stuff
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 128

# Rescale multplies your values by the value you pass it.
# Here, we're taking the 0-255 RGB values and changing them to 0-1
train_datagen = ImageDataGenerator(rescale=1./255)
validate_datagen = ImageDataGenerator(rescale=1./255)

# This will indefinitely generate batches of augmented image data
train_generator = train_datagen.flow_from_directory(
    'cvd_data/train',  # pull from this directory
    target_size=(150, 150),  # all images will be resized to 150x150
    batch_size=batch_size,
    class_mode='binary') # because we have two classes

# Same generator, for validation data
validation_generator = validate_datagen.flow_from_directory(
    'cvd_data/validation',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary')


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Monitor our validation loss for early stopping.
# Stop if we haven't improved in 2 epochs
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)]

# Notice that we have to use fit_generator when using the data generator
model.fit_generator(
    train_generator,
    steps_per_epoch=20000 // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=5000 // batch_size,
    callbacks=callbacks )

model.save('model1.h5') 

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/50
156/156 [==============================] - 46s 296ms/step - loss: 0.6755 - acc: 0.5899 - val_loss: 0.5826 - val_acc: 0.6755
Epoch 2/50
156/156 [==============================] - 44s 283ms/step - loss: 0.5539 - acc: 0.7144 - val_loss: 0.5116 - val_acc: 0.7450
Epoch 3/50
156/156 [==============================] - 44s 283ms/step - loss: 0.5011 - acc: 0.7588 - val_loss: 0.4482 - val_acc: 0.7871
Epoch 4/50
156/156 [==============================] - 44s 281ms/step - loss: 0.4558 - acc: 0.7892 - val_loss: 0.4274 - val_acc: 0.7961
Epoch 5/50
156/156 [==============================] - 44s 281ms/step - loss: 0.4245 - acc: 0.8077 - val_loss: 0.4090 - val_acc: 0.8125
Epoch 6/50
156/156 [==============================] - 44s 280ms/step - loss: 0.3942 - acc: 0.8225 - val_loss: 0.3949 - val_acc: 0.8295
Epoch 7/50
156/156 [==============================] - 43s 279ms/step - loss: 0.3673 - acc: 0.8395 - val_l

### 1st Model results

|Model|High Train Acc|End Train Acc|High Val Acc|End Val Acc |
|-----|--------------|-------------|------------|------------|
|1    |87.95%        |87.95%       |84.50%      |83.23%      |

- Our numbers aren't as high as I'd like, but we actually would've ranked [87th out of 215 in the original Kaggle competetion](https://www.kaggle.com/c/dogs-vs-cats/leaderboard)

Let's move on to generated data and see how they compare.

## Time to tweak some images

We're going to build another model using the same neural netowrk as before, but with 10% of the data.

We need to:
1. See what data augmentation looks like
2. Create our smaller dataset
3. Train our model on the augmented, smaller dataset.

Let's see what this augmentaiton really looks like.

In [4]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


if os.path.exists('example'):
    for pic in os.listdir('example'):
        os.remove(f'example/{pic}')
else:
    os.makedirs('example')

datagen = ImageDataGenerator(
    # Range of degrees to randomly rotate pics
    rotation_range=40, 
    # Fraction that we will randomly translate pictures vertically or horizontally
    width_shift_range=0.2,
    height_shift_range=0.2,
    # Randomly shear (distort diagonally)
    shear_range=0.2,
    zoom_range=0.2,
    # Flip pic right and left
    horizontal_flip=True,
    # Some of these operations create new pixels.
    # fill_mode fills these new pixels in
    fill_mode='nearest')

img = load_img('cvd_data/train/dogs/dog.5.jpg')
x = img_to_array(img)
# Our image is a 3x150x150 array.
# Making it a 1x3x150x150 is essentially saying "there's only one image here" 
x = x.reshape((1,) + x.shape)

# Create batches of transformed images
# Saves the results to the `example/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='example', save_prefix='dog', save_format='jpeg'):
    i += 1
    if i > 5:
        break  # otherwise the generator would loop indefinitely

os.listdir('example')

['dog_0_1217.jpeg',
 'dog_0_1938.jpeg',
 'dog_0_502.jpeg',
 'dog_0_7326.jpeg',
 'dog_0_8114.jpeg',
 'dog_0_9671.jpeg']

#### And here's our images:

Example Augmented Pics 1      |2                             |3
:----------------------------:|:----------------------------:|:----------------------------:
![](examples/dog_0_7591.jpeg) |![](examples/dog_0_8444.jpeg) |![](examples/dog_0_62.jpeg)
![](examples/dog_0_2581.jpeg) |![](examples/dog_0_2395.jpeg) |![](examples/dog_0_3271.jpeg)

In order to understand why these relatively subtle changes work, think back to how convolutional layers work. They figure out shapes. None of the above pictures are so distorted that the shapes a wrong, they just allow the convolutional layer to see, for example, the shape of the dog's ears in multiple angles and sizes.

### Why so much data?

Time to axe some images! Both training and validation will be 10% of their former glory.

In [5]:
import os
import shutil

def get_path(index):
    if index < 1000:
        return 'train'
    if index > 9749:
        return 'validation'

if not os.path.exists('cvd_data/small'):
    # Create dir structure
    os.makedirs('cvd_data/small')
    os.makedirs('cvd_data/small/train/cats')
    os.makedirs('cvd_data/small/train/dogs')
    os.makedirs('cvd_data/small/validation/cats')
    os.makedirs('cvd_data/small/validation/dogs')

    dog_pics = os.listdir('cvd_data/train/dogs')
    cat_pics = os.listdir('cvd_data/train/cats')

    for pic_list in [dog_pics, cat_pics]:
        for pic in pic_list:
            index = pic_list.index(pic)
            # Sorts first 1K into train and last 250 into validation
            path = get_path(index)
            if path:
                label = 'dogs' if 'dog' in pic else 'cats'
                source = f'cvd_data/train/{label}/{pic}'
                dest = f'cvd_data/small/{path}/{label}/{pic}'
                shutil.copy(source, dest)  

train_pics = len(os.listdir('cvd_data/small/train/dogs') + os.listdir('cvd_data/small/train/cats'))
val_pics = len(os.listdir('cvd_data/small/validation/dogs') + os.listdir('cvd_data/small/validation/cats'))
print(f'Training Pics: {train_pics}')
print(f'Validation Pics: {val_pics}')

Training Pics: 2000
Validation Pics: 500


## Model 2 Time

This is mostly the same code as before. The big difference is the train_datagen.

Note that we're changing `steps_per_epoch` because this number is dependant on the number of samples.

I'm also removing early stopping for now. I'm not sure how this model will run and want to make sure we aren't stopping it too early.

I'm not sure if overfitting will be an issue. On the one hand, the model shouldn't see the same image twice. On the other hand, the augmented images it sees are coming from a small set of 'root' images. 

In [5]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 128

# We don't want to mess with our validation data, but this is where we do all of our augmentaiton
# Notice that we cut the options that resulted in blurry edges.
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

validate_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'cvd_data/small/train',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = validate_datagen.flow_from_directory(
    'cvd_data/small/validation',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary')


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit_generator(
    train_generator,
    steps_per_epoch=2000 // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=500 // batch_size)

model.save('model2.h5') 

Found 2000 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
Epoch 1/50
15/15 [==============================] - 10s 653ms/step - loss: 0.8194 - acc: 0.5115 - val_loss: 0.6896 - val_acc: 0.5026
Epoch 10/50
15/15 [==============================] - 9s 604ms/step - loss: 0.5994 - acc: 0.6870 - val_loss: 0.6194 - val_acc: 0.6432
Epoch 20/50
15/15 [==============================] - 9s 606ms/step - loss: 0.5355 - acc: 0.7276 - val_loss: 0.6422 - val_acc: 0.6901
Epoch 30/50
15/15 [==============================] - 9s 597ms/step - loss: 0.4573 - acc: 0.7855 - val_loss: 0.6053 - val_acc: 0.7422
Epoch 40/50
15/15 [==============================] - 9s 598ms/step - loss: 0.3989 - acc: 0.8183 - val_loss: 0.5988 - val_acc: 0.7422
Epoch 50/50
15/15 [==============================] - 9s 627ms/step - loss: 0.3725 - acc: 0.8250 - val_loss: 0.5732 - val_acc: 0.7656


### 2nd Model results

|Model|High Train Acc|End Train Acc|High Val Acc|End Val Acc |
|-----|--------------|-------------|------------|------------|
|1    |87.95%        |87.95%       |84.50%      |83.23%      |
|2    |84.44%        |82.50%       |76.56%      |76.56%      |

I think this actually turned out well. We were only 7% lower than the first model and we would've placed [99th out of 215 in the original Kaggle competetion](https://www.kaggle.com/c/dogs-vs-cats/leaderboard). Not bad for only using 10% of the data!

# Using Pre-trained Models

We can take general purpose models, chop the bits off we don't need, and add our own pieces in. Remember, Convolutional Neural Nets identify patterns and shapes in images, then feed their outputs into fully connected (dense) layers that do the actual classifying. This means that we can take a CNN that others have provided, remove the fully connected layers at the end, and add our own classifier.

Our pre-trained model was trained on the ImageNet dataset. This is the most famous computer vision dataset available. Since 2010 there has been the yearly ImageNet Large Scale Visual Recognition Challenge (ILSVRC) where contestants train on over 14 million images and try to identify 1000 different classes.

Most important for us, those 1000 classes include over 100 specific breeds of cats and dogs. Neural Networks trained on this dataset include convolutional layers that can easily identify the visual characteristics that make up a dog or cat.

Specifically, we're going to be using the VGG16 architecture. This model was developed in 2014 by the Visual Geometry Group (VGG) from Oxford. The '16' is the number of working layers (Convolutional and Dense). There are larger versions, but 16 should be more than enough for us.

Here's what VGG16 looks like: 

![VGG16](https://www.researchgate.net/profile/Kasthurirangan_Gopalakrishnan/publication/319952138/figure/fig2/AS:613973590282251@1523394119133/A-schematic-of-the-VGG-16-Deep-Convolutional-Neural-Network-DCNN-architecture-trained.png)

It's those end layers (highlighted in red) that we'll be removing. The layers that it comes with by default are going to output an answer that assumes we want 1 of 1000 classes. We're just looking for 1 of 2.

I should note that we're actually breaking this up into two models. We're going to use VGG16 (minus the calssifier) to processes our images then save the features that it outputs to a file. We're then going to feed those files into our own, custom classifier.

The odd part about this is that we're taking something capable of identifying 1000 different classes and multiple breeds of cats and dogs and severely reducing its functionality down to just say 'Cat' or 'Dog'. But think about what this can be used for: You can use the exact technique laid out in this notebook to make any kind of visual classifier that you need.

I've broken the feature array generation up from the classifier training. The feature generation is a massive operation. The numpy arrays that it generates and saves can put serious strain on your computer. Uncomment the `datagen()` function call in the 2nd cell if you want to run this for yourself.

If you're really following along in detail, or if you've looked at the code in the website associated with this model, you might have noticed some differences in the below code and the deployed model. I had a really difficult time getting the code working in JavaScript. At one point I thought the model might be the problem (it wasn't) so I started playing with it here.

- The only difference between this code and the deployed code is that the model for the website outputs an array of 2 numbers, with the 1 identifying the class([0, 1] vs [1, 0]). This code outputs a single number with a 0 for Cat and a 1 for dog. I thought this may have been an issue, but it wasn't. I did end up just liking the two number system as a personal preference, so I kept it in the deployed model.
    - I'm going to leave in comments about those changes to give you an idea of what needed to change. Any comment marked `[2Output]` shows what I would've used for this change.
    
- Another change I tried was increasing the image size from 150x150 to 224x224. I didn't end up keeping this change, but it did increase the end accuracy from 94.83% to 96.47%. The model this made was way larger and would've slowed the site down. 
    - Comments showing this change will be marked `[LargeInput]`

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# That's right: Keras has these models built in
from tensorflow.keras.applications.vgg16 import VGG16

def datagen():

    train_datagen = ImageDataGenerator(
        rescale=1.,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

    validate_datagen = ImageDataGenerator(rescale=1.)

    # Here's were we load the pre-trained model
    # - Without the dense layers at the end (the 'top')
    # - With all of the imagenet trained weights
    vgg16 = VGG16(include_top=False, weights='imagenet')

    # Notice that I'm using the full dataset AND data transformation.
    train_generator = train_datagen.flow_from_directory(
        'cvd_data/train',
        #[LargeInput] target_size=(224, 224),
        target_size=(150, 150),
        batch_size=batch_size,
        # We are actually trying to classify here, so we're switching class_mode to none
        class_mode=None,
        # We're feed in all of cats, then all of the dogs. Reasons will be clear soon
        shuffle=False)

    validation_generator = validate_datagen.flow_from_directory(
        'cvd_data/validation',
        #[LargeInput] target_size=(224, 224),
        target_size=(150, 150),
        batch_size=batch_size,
        # Same as last generator
        class_mode=None,
        shuffle=False)

    # This runs all of our traning data through VGG16
    # We're not getting predictions. We're getting giant
    #   numpy arrays that represent the features extraced by VG16.
    # Notice that we're not doing any training here. We're just asking for predictions.
    features_train = vgg16.predict_generator(train_generator, train_samples // batch_size)
    # We're saving those features to a file
    np.save('features_train.npy', features_train)

    # Same as traning data
    features_validation = vgg16.predict_generator(validation_generator, validation_samples // batch_size)
    np.save('features_validation.npy', features_validation)
    
    print('Done data-gen-ing')

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.applications.vgg16 import VGG16

batch_size = 32
train_samples = 20000
validation_samples = 5000

# Uncomment the below function call to actually generate this data.
# Warning: My comp has 16GB of memory and a 1080Ti and it stuggled
#datagen()

# Loading our feature file
train_data = np.load('features_train.npy')
# We need to create labels for all of our data
# Remember how we didn't shuffle the data? This is why.
# This just makes an array that the same length as our data
#    that's all 0's followed by all 1's.
#[2Output] train_labels = np.array([0, 1] * int(len(train_data) / 2) + [1, 0] * int(len(train_data) / 2))
train_labels = np.array([0] * int(len(train_data) / 2) + [1] * int(len(train_data) / 2))
validation_data = np.load('features_validation.npy')
#[2Output] validation_labels = np.array([0, 1] * int(len(validation_data) / 2) + [1, 0] * int(len(validation_data) / 2))
validation_labels = np.array([0] * int(len(validation_data) / 2) + [1] * int(len(validation_data) / 2))

# Early stopping, so we don't have to worry about as much overfitting.
# Remember: We're feeding in feautures from a pretty BA CNN. 
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]

model = Sequential()
# Same as our old CNN models: We're getting 3D features that need to be flattened to 1D
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
#[2Output] model.add(Dense(2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#[2Output] model.compile(loss=keras.losses.categorical_crossentropy,
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=100,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels),
          callbacks=callbacks)

model.save('model3.h5')

Train on 20000 samples, validate on 4992 samples
Epoch 1/100
20000/20000 [==============================] - 32s 2ms/step - loss: 1.7667 - acc: 0.8849 - val_loss: 1.9369 - val_acc: 0.8778
Epoch 2/100
20000/20000 [==============================] - 32s 2ms/step - loss: 1.2438 - acc: 0.9206 - val_loss: 0.7781 - val_acc: 0.9499
Epoch 3/100
20000/20000 [==============================] - 32s 2ms/step - loss: 1.0961 - acc: 0.9306 - val_loss: 0.7030 - val_acc: 0.9551
Epoch 4/100
20000/20000 [==============================] - 32s 2ms/step - loss: 1.0263 - acc: 0.9353 - val_loss: 0.5260 - val_acc: 0.9667
Epoch 5/100
20000/20000 [==============================] - 32s 2ms/step - loss: 0.9295 - acc: 0.9412 - val_loss: 0.5622 - val_acc: 0.9643
Epoch 6/100
20000/20000 [==============================] - 32s 2ms/step - loss: 0.9625 - acc: 0.9392 - val_loss: 0.6274 - val_acc: 0.9603
Epoch 7/100
20000/20000 [==============================] - 31s 2ms/step - loss: 0.8461 - acc: 0.9465 - val_loss: 0.5821 - v

## Model 3 Results

|Model       |High Train Acc|End Train Acc|High Val Acc|End Val Acc |
|------------|--------------|-------------|------------|------------|
|1           |87.95%        |87.95%       |84.50%      |83.23%      |
|2           |84.44%        |82.50%       |76.56%      |76.56%      |
|3           |93.71%        |93.61%       |95.01%      |94.83%      |
|224Input    |95.19%        |95.19%       |96.67%      |96.47%      |

For model3: In only 14 epochs we acheived 94.83% validation accuracy. You should also note that this model trained much, much faster that model's 1 and 2 because there were fewer layers overall and no convolutional layers to train.

For 224Input: We got an even higher accuracy in even less time with higher resolution inputs. But our model files ended up being larger. 

# Tensorboard

In most of my notebooks so far we've been either guessing at, or using someone else's neural network architecture. Tensorboard allows us to get insights into our network and its training so we can further refine it. 

This works by adding a callback, just like early stopping. The tensorboard callback requires a logging directory. We'll then take those logs and feed them into the `tensorboard` command, which comes installed with tensorflow by default.

I'm leaving the tensorboard logs in the repo. Just run `tensorboard --logdir=logs/` from the P4 directory. This will output a webpage for your usage. The default will be `http://localhost:6006`.

Unfortunately, Model 3 isn't great for tensorboard. The actual model that we're training is small and boring. Here, we're going to run Model 1 again, but with early stopping set to `patience=0` and the `TensorBoard` callback. 

Note that you can run tensorboard while a model is training.

In [2]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 128

train_datagen = ImageDataGenerator(rescale=1./255)
validate_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'cvd_data/train',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = validate_datagen.flow_from_directory(
    'cvd_data/validation',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='binary')


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=0),
    # This is really all there is to making TensorBoard work.
    keras.callbacks.TensorBoard(log_dir='./logs')
]

model.fit_generator(
    train_generator,
    steps_per_epoch=20000 // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=5000 // batch_size,
    callbacks=callbacks )

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/50
156/156 [==============================] - 46s 298ms/step - loss: 0.6716 - acc: 0.6040 - val_loss: 0.5858 - val_acc: 0.7063
Epoch 2/50
156/156 [==============================] - 45s 287ms/step - loss: 0.5640 - acc: 0.7126 - val_loss: 0.4989 - val_acc: 0.7654
Epoch 3/50
156/156 [==============================] - 46s 298ms/step - loss: 0.4990 - acc: 0.7614 - val_loss: 0.4403 - val_acc: 0.7903
Epoch 4/50
156/156 [==============================] - 45s 291ms/step - loss: 0.4650 - acc: 0.7849 - val_loss: 0.4230 - val_acc: 0.8095
Epoch 5/50
156/156 [==============================] - 44s 285ms/step - loss: 0.4266 - acc: 0.8055 - val_loss: 0.4009 - val_acc: 0.8195
Epoch 6/50
156/156 [==============================] - 46s 292ms/step - loss: 0.3896 - acc: 0.8278 - val_loss: 0.4033 - val_acc: 0.8203


Here's some example graphs that you'll see:

This is a visualization of our network. You're able to click on each node in the graph and see the tensor operations inside.

<img src="examples/tb_1.PNG" width="800" />

This is a visualization of our network's training performance. Note the other graphs that can be expanded at the bottom:

<img src="examples/tb_2.PNG" width="800" />

Honestly, the really fun parts of tensorflow (the what-if tool, for example) require using tensorflow serving to serve the models and play with the data. This is a bit more on the production end of things.

## Exporting for use in JavaScript

So we have some models, but how do we use them? I'm not going to get into the details of what you need to do in JavaScript to make this work, but here's how you take a Keras model, generated by Python, and convert it for use in TensorflowJS:

```bash
#Install the tensorflowjs python utility
pip install tensorflowjs

# Run the converter command. This is going to output multiple files into a directory
tensorflowjs_converter --input_format keras <MY_MODEL>.h5 <OUTPUT_DIR>
```

That really all you have to do to convert the model. You can host your models locally or in the cloud. I'm using S3 for my site.

Here's how you import the model for use in your javascript:

```bash
# Install tensorflowjs via npm
npm install @tensorflow/tfjs
```

Then...

```javascript
# Import tensorflow and load your model
import * as tf from '@tensorflow/tfjs';
let model = tf.loadModel('https://s3-us-west-2.amazonaws.com/testing-models/catvsdog_classifier/model.json')
# Make a prediction
let prediction = model.predict(features)
```

Look familiar?
